# Generate CSVs from COVID-19 WHO data

## Dependencies

In [1]:
import pandas as pd
import os
from datetime import datetime

INPUT_PATH = 'input/'
OUTPUT_PATH = 'public/data/'

REGIONS_CSV_PATH = os.path.join(INPUT_PATH, 'regions.csv')
DEATHS_CSV_PATH = os.path.join(INPUT_PATH, 'deaths.csv')
CASES_CSV_PATH = os.path.join(INPUT_PATH, 'cases.csv')

In [2]:
!mkdir -p $OUTPUT_PATH

## Load the data

In [3]:
df_regions = pd.read_csv(REGIONS_CSV_PATH)

Transform the "wide" format into "long" format, which is easier to work with.

In [4]:
def melt_csv(df, var_name):
    return df.melt(
        id_vars=df.columns[0], 
        value_vars=df.columns[1:], 
        var_name='location', 
        value_name=var_name
    ).dropna()

In [5]:
df_deaths = melt_csv(pd.read_csv(DEATHS_CSV_PATH, header=1).rename(columns={ 'Date': 'date' }), 'total_deaths')
df_cases = melt_csv(pd.read_csv(CASES_CSV_PATH, header=1).rename(columns={ 'Date': 'date' }), 'total_cases')

Convert all numbers from floating point to integers:

In [6]:
df_deaths['total_deaths'] = df_deaths['total_deaths'].astype('Int64')
df_cases['total_cases'] = df_cases['total_cases'].astype('Int64')

## Calculations

Join cases & deaths into one dataframe

In [7]:
df_merged = df_cases.merge(
    df_deaths, 
    how='outer', 
    left_on=['date', 'location'], 
    right_on=['date', 'location']
).sort_values(by=['location', 'date'])

Standardize names to OWID names

In [8]:
df_regions_merged = df_regions.merge(
    df_merged[['location']].drop_duplicates(),
    how="outer",
    left_on="WHO Country Name",
    right_on="location"
)

In [9]:
assert(df_regions_merged['OWID Country Name'].isnull().any() == False)

In [10]:
who_name_replace_map = { r['WHO Country Name']: r['OWID Country Name'] for r in df_regions_merged.to_dict('records') }

In [11]:
df_merged['location'] = df_merged['location'].replace(who_name_replace_map)

Calculate daily cases & deaths

In [12]:
df_merged['new_cases'] = df_merged.groupby('location')['total_cases'].diff().astype('Int64')
df_merged['new_deaths'] = df_merged.groupby('location')['total_deaths'].diff().astype('Int64')

Create a `Worldwide` aggregate

In [13]:
df_global = df_merged.groupby('date').sum().reset_index()
df_global['location'] = 'Worldwide'

In [14]:
df_merged = pd.concat([df_merged, df_global], sort=True)

Calculate doubling rates

In [15]:
def get_days_to_double(df, col_name):
    try:
        # verbose because being very careful not to modify original data with dates
        latest = df.loc[pd.to_datetime(df['date']).idxmax()]
        df_lt_half = df[df[col_name] <= (latest[col_name] / 2)]
        half = df_lt_half.loc[pd.to_datetime(df_lt_half['date']).idxmax()]
        return (pd.to_datetime(latest['date']) - pd.to_datetime(half['date'])).days
    except:
        return None

In [16]:
days_to_double_cases = pd.DataFrame([
    (loc, get_days_to_double(df, 'total_cases')) 
    for loc, df in df_merged.groupby('location')
], columns=['location', 'days_to_double_cases'])
days_to_double_cases['days_to_double_cases'] = days_to_double_cases['days_to_double_cases'].astype('Int64')

## Inspect the results

In [17]:
df_merged

,date,location,new_cases,new_deaths,total_cases,total_deaths
1925,2020-02-25,Afghanistan,NaN,NaN,1,NaN
1926,2020-02-26,Afghanistan,0,NaN,1,NaN
1927,2020-02-27,Afghanistan,0,NaN,1,NaN
1928,2020-02-28,Afghanistan,0,NaN,1,NaN
1929,2020-02-29,Afghanistan,0,NaN,1,NaN
1930,2020-03-01,Afghanistan,0,NaN,1,NaN
1931,2020-03-02,Afghanistan,0,NaN,1,NaN
1932,2020-03-03,Afghanistan,0,NaN,1,NaN
1933,2020-03-04,Afghanistan,0,NaN,1,NaN
1934,2020-03-05,Afghanistan,0,NaN,1,NaN


In [18]:
df_merged[df_merged['location'] == 'Worldwide']

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2020-01-21,Worldwide,0,0,282,6
1,2020-01-22,Worldwide,32,0,314,6
2,2020-01-23,Worldwide,266,11,581,17
3,2020-01-24,Worldwide,262,8,846,25
4,2020-01-25,Worldwide,467,16,1320,41
5,2020-01-26,Worldwide,691,15,2014,56
6,2020-01-27,Worldwide,783,24,2798,80
7,2020-01-28,Worldwide,1792,26,4593,106
8,2020-01-29,Worldwide,1468,26,6065,132
9,2020-01-30,Worldwide,1750,38,7818,170


## Write output files

In [20]:
df_merged.to_csv(os.path.join(OUTPUT_PATH, 'full_data.csv'), index=False)

In [21]:
for col_name in ['total_cases', 'total_deaths', 'new_cases', 'new_deaths']:
    df_pivot = df_merged.pivot(index='date', columns='location', values=col_name)
    # move Worldwide to first column
    cols = df_pivot.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Worldwide')))
    df_pivot[cols].to_csv(os.path.join(OUTPUT_PATH, '%s.csv' % col_name))

In [22]:
days_to_double_cases.to_csv(os.path.join(OUTPUT_PATH, 'days_to_double_cases.csv'), index=False)

In [23]:
df_regions.to_csv(os.path.join(OUTPUT_PATH, 'regions.csv'), index=False)

Create `grapher.csv`

In [24]:
df_grapher = df_merged.copy()
df_grapher['date'] = pd.to_datetime(df_grapher['date']).map(lambda date: (date - datetime(2020, 1, 21)).days)

In [25]:
df_grapher[['location', 'date', 'new_cases', 'new_deaths', 'total_cases', 'total_deaths']] \
    .rename(columns={
        'location': 'country',
        'date': 'year',
        'new_cases': 'Daily new confirmed cases of COVID-19',
        'new_deaths': 'Daily new confirmed deaths due to COVID-19',
        'total_cases': 'Total confirmed cases of COVID-19', 
        'total_deaths': 'Total confirmed deaths due to COVID-19'
    }) \
    .to_csv(os.path.join(OUTPUT_PATH, 'grapher.csv'), index=False)